## CSV 불러오기

In [3]:
import pandas as pd
import numpy as np

actual_weather_path = './SURFACE_ASOS_201_HR_2023_2023_2024.csv'
forecast_rain_path = './불은면_1시간강수량_20230101_20231231.csv'
forecast_temp_path = './불은면_1시간기온_20230101_20231231.csv'
forecast_snow_path = './불은면_1시간적설_20230101_20231231.csv'
forecast_rain_type_path = './불은면_강수형태_20230101_20231231.csv'
forecast_rain_prob_path = './불은면_강수확률_20230101_20231231.csv'
forecast_rain_humidity_path = './불은면_습도_20230101_20231231.csv'
forecast_rain_speed_path = './불은면_풍속_20230101_20231231.csv'
forecast_wind_direction_path = './불은면_풍향_20230101_20231231.csv'
forecast_status_path = './불은면_하늘상태_20230101_20231231.csv'

actual_weather = pd.read_csv(actual_weather_path, encoding='cp949')
forecast_rain = pd.read_csv(forecast_rain_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '강수량'])
forecast_temp = pd.read_csv(forecast_temp_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '기온'])
forecast_snow = pd.read_csv(forecast_snow_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '적설'])
forecast_rain_type = pd.read_csv(forecast_rain_type_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '강수형태'])
forecast_rain_prob = pd.read_csv(forecast_rain_prob_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '강수확률'])
forecast_rain_humidity = pd.read_csv(forecast_rain_humidity_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '습도'])
forecast_rain_speed = pd.read_csv(forecast_rain_speed_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '풍속'])
forecast_wind_direction = pd.read_csv(forecast_wind_direction_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '풍향'])
forecast_status = pd.read_csv(forecast_status_path, encoding='cp949', header=0, names=['날짜', '시간', '예보시간', '하늘상태'])

print("실제 기상 데이터\n", actual_weather.head())
# print("\n1시간 강수량 데이터\n", forecast_rain.head())
# print("\n1시간 기온 데이터\n", forecast_temp.head())
# print("\n1시간 적설량 데이터\n", forecast_snow.head())
# print("\n강수 형태 데이터\n", forecast_rain_type.head())
# print("\n강수 확률 데이터\n", forecast_rain_prob.head())
# print("\n하늘 상태 데이터\n", forecast_status.head())

실제 기상 데이터
     지점                일시  기온(°C)  강수량(mm)  풍속(m/s)  풍향(16방위)  습도(%)  증기압(hPa)  \
0  201  2023-01-01 00:00     3.2      NaN      2.8     290.0     69       5.3   
1  201  2023-01-01 01:00     2.9      NaN      2.3     270.0     66       5.0   
2  201  2023-01-01 02:00     2.1      NaN      2.4     270.0     71       5.0   
3  201  2023-01-01 03:00     2.0      NaN      1.1     180.0     65       4.6   
4  201  2023-01-01 04:00     0.7      NaN      1.1     200.0     57       3.7   

   이슬점온도(°C)  현지기압(hPa)  ...  운형(운형약어)  최저운고(100m )  시정(10m)  지면상태(지면상태코드)  \
0       -1.9     1024.2  ...       NaN          NaN    799.0           NaN   
1       -2.8     1023.8  ...       NaN         75.0    749.0           NaN   
2       -2.6     1024.0  ...       NaN         15.0    675.0           NaN   
3       -3.9     1024.3  ...       NaN         11.0    800.0           NaN   
4       -6.8     1024.4  ...       NaN         13.0   1346.0           NaN   

   현상번호(국내식)  지면온도(°C)  5cm 지중온도(

## forecast_weather 데이터 전처리

In [4]:
# 각 데이터프레임에 날짜와 시간 예보시간 기준으로 합치기
forecast_weather = pd.concat([forecast_rain, forecast_temp, forecast_snow, forecast_rain_type, forecast_rain_prob,
                           forecast_rain_humidity, forecast_rain_speed, forecast_wind_direction, forecast_status],
                          axis=1)

fw_df = pd.DataFrame(forecast_weather)

# 중복되는 날짜와 시간 컬럼 제거 (첫 번째 데이터프레임 기준으로 유지)
fw_df = fw_df.loc[:, ~fw_df.columns.duplicated()]

# forecast_weather를 '통합예보데이터.csv' 파일로 저장
fw_df.to_csv('통합예보데이터.csv', index=False, encoding='cp949')

### 날짜

In [5]:
# 2023년 1월 1일 데이터에만 없는 날짜 안내 행 추가
# 빈 문자열을 NaN으로 변경하여 할당 ("" 빈 문자열을 넣었다가 데이터 타입 불일치 오류 발생 -> NaN으로 해결)
fw_df.loc[0] = [' Start : 20230101 ', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

# 결과 확인
#print(fw_df.head())

### 날짜: Start:20230102 형식으로 중간중간 문자열 데이터 존재 -> 년월일 추출해 변환 2023-01-01
# Start 행 날짜 형식으로 변경
for day in fw_df['날짜']:
  if "Start" in day:
    date_str = day.split(":")[1].strip()
    # 날짜 형식 변경: "20231003" -> "2023-10-03"
    formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    #print(formatted_date)
    fw_df.loc[fw_df['날짜'] == day, '날짜'] = formatted_date

# 날짜 열에서 숫자형이거나 다른 형식은 NaN으로 변환
fw_df['날짜'] = fw_df['날짜'].apply(lambda x: x if isinstance(x, str) and len(x) == 10 else np.nan)

# 숫자 행을 이전 행인 변경한 날짜로 채우기
fw_df['날짜'] = fw_df['날짜'].ffill()

### 시간

In [7]:
### 시간: 2시 200 으로 표기 -> 2:00 시간 형식으로 변환
fw_df['시간'] = fw_df['시간'].apply(
    lambda x: f"{int(x) // 100:02}:{int(x) % 100:02}" if pd.notnull(x) and x != '' else x
    )
#print(fw_df['시간'])

### 변경했던 Start 날짜 행 제거하기(NaN이 10개 이상일 경우)
fw_df = fw_df[fw_df.isna().sum(axis=1) < 10]

print(fw_df)

                날짜     시간  예보시간  강수량   기온   적설  강수형태  강수확률    습도   풍속     풍향  \
1       2023-01-01  02:00   7.0  0.0 -4.0  0.0   0.0   0.0  50.0  4.2  318.0   
2       2023-01-01  02:00   8.0  0.0 -4.0  0.0   0.0  20.0  55.0  4.6  318.0   
3       2023-01-01  02:00   9.0  0.0 -5.0  0.0   0.0  20.0  55.0  4.8  320.0   
4       2023-01-01  02:00  10.0  0.0 -6.0  0.0   0.0  30.0  60.0  5.0  322.0   
5       2023-01-01  02:00  11.0  0.0 -6.0  0.0   0.0  20.0  60.0  5.0  325.0   
...            ...    ...   ...  ...  ...  ...   ...   ...   ...  ...    ...   
185779  2023-12-31  23:00  60.0  0.0 -1.0  0.0   0.0   0.0  85.0  3.3  315.0   
185780  2023-12-31  23:00  61.0  0.0 -2.0  0.0   0.0   0.0  85.0  2.9  316.0   
185781  2023-12-31  23:00  62.0  0.0 -2.0  0.0   0.0   0.0  85.0  2.5  318.0   
185782  2023-12-31  23:00  63.0  0.0 -3.0  0.0   0.0   0.0  85.0  2.1  323.0   
185783  2023-12-31  23:00  64.0  0.0  NaN  0.0   0.0   0.0  85.0  1.9  328.0   

        하늘상태  
1        1.0  
2        

### 일시

In [10]:
### 일시: 날짜 + 시간 -> datetime 형식으로 결합 2023-01-01 2:00
fw_df['일시'] = fw_df['날짜'].astype(str) + ' ' + fw_df['시간']
fw_df['일시'] = pd.to_datetime(fw_df['일시'], format='%Y-%m-%d %H:%M')
print(fw_df['일시'])

# forecast_weather를 '통합예보데이터.csv' 파일로 저장
fw_df.to_csv('통합예보데이터.csv', index=False, encoding='cp949')

1        2023-01-01 02:00:00
2        2023-01-01 02:00:00
3        2023-01-01 02:00:00
4        2023-01-01 02:00:00
5        2023-01-01 02:00:00
                 ...        
185779   2023-12-31 23:00:00
185780   2023-12-31 23:00:00
185781   2023-12-31 23:00:00
185782   2023-12-31 23:00:00
185783   2023-12-31 23:00:00
Name: 일시, Length: 185419, dtype: datetime64[ns]
